<p style="text-align:center"><font size="10" >Kawase Seminar 05-2</font></p>

全2回にわけて実際の論文の手法を実践し、スクレイピングとテキストマイングの利用を体験していきます。

そんな第5, 6回では、1.実際に現状を把握する, 2.解くための筋道を立てる, 3.Densuke(田村)が実装したコードを読んで2の筋道が実際に正しかったのか確認する, といったことを逐次行っていきます。

今回以降は、時間の関係上実装はそこまで行えませんがかなりの学びになるので、皆さんも各自でトライして頂けると良いと思われます💪

### **やること**
-  今回実装する論文の手法を把握する
-  スクレイピング分野における実装を大まかに考える
-  大まかな実装を細かい単位に分解し具体的なロジックを組み立てる
-  **ロジックをもとに実装する ->イマココ**


今回のコードを読む目的としては、どのように田村が実装したのかを田村の思考を理解し、

実際に自分で書く際の糧になってもらいことにあります。


---
### **田村のTODOメモ**
1. 環境構築
 1. pip install　でseleniumなどのスクレイピングライブラリを取得
 2. chrome driverが必要な場合、google colabのOSにインストールする
 
 
2. 実装(プログラムの動き)
 1. 取得したい企業のスレッドにアクセスする
 2. 最下部までメッセージをスクロールする
 3. クローリングでメッセージデータを取得する
 4. 全てのメッセージを取得したら次のスレッドに行く
 
 5. 1~4を繰り返し、取得したい最初の日付から最後に日付までのデータを取得できたらプログラムを終了する
 6. 取得したメッセージデータをpandasDataFrameに格納し、csv形式で保存する


## 1. 環境構築

**1.1 pip install　でseleniumなどのスクレイピングライブラリを取得**


**1.2 chrome driverが必要な場合、google colabのOSにインストールする**

In [11]:
#seleniumのインストール
# reference https://enjoy-a-lot.com/google-colaboratory-selenium/#toc1
!pip install selenium

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.4MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Get:12 http://archive.ub


## 2. 実装(プログラムの動き)

### 試しに動くか確認する

In [22]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://enjoy-a-lot.com/")  
title = driver.find_element_by_class_name('logo-header')
print(title.text)

web_log


  **2.2 最下部までメッセージをスクロールする**

ページスクロールの対策
https://teratail.com/questions/122242

In [23]:
import time

from selenium import webdriver

# Google Drive起動
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

# yahoo表示
driver.get('https://finance.yahoo.co.jp/cm/message/1004684/a5aaa1bca5sa5ca5af')

# javascriptを実行してページの最下部へ移動
# https://teratail.com/questions/122242
html01=driver.page_source
count = 0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    count += 1
    print(count)
    html02=driver.page_source
    if html01!=html02:
        html01=html02
    else:
        break

# https://www.mittsu-kosen.com/%E3%80%90pythonxselenium%E3%80%91web%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E3%81%A7%E3%82%88%E3%81%8F%E4%BD%BF%E3%81%86%E6%93%8D%E4%BD%9C%E3%83%A1%E3%82%BD%E3%83%83/#i-2
test = driver.find_elements_by_class_name("comText")


print("終了しました")
test

1
2
3
4
5
6
7
8
9
10
11
終了しました


[<selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="f0fafc13-f5ac-4a32-bf39-f393936658c1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="a441e051-c056-4317-9e2c-f6b907668ef8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="a173c565-5ce9-4136-be60-a631299b7dba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="7e2bdf7f-b2de-477f-8e6c-b3d528221f75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="b21d5994-b975-4c68-91c0-241fca43983b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="fa507d68-606d-4587-aec2-f8bc49b4c0a0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a3a9b89cce4a12cc90c80841ad7a5fe2", element="1a731c44-0d3e-412b-848a-72

 **2.3 クローリングでメッセージデータ(日付とメッセージ)を取得する**


In [24]:
import re
com_text_values = []
com_date_values = []
 
com_text = driver.find_elements_by_class_name("comText")

for idx in range(len(com_text)):
  com_text_values.append(com_text[idx].text)
  tmp = driver.find_elements_by_class_name("comment")[idx].find_elements_by_tag_name("a")
  for val in tmp :
    if len(re.findall('.*月.*日*', val.text)) :
      com_date_values.append(val.text)
      break
print(len(com_text_values), len(com_date_values))  

213 213


試しにデータをDataFrame型に入れてみる

In [25]:
import pandas as pd
fetched_data = pd.DataFrame()
fetched_data["Date"] =  com_date_values
fetched_data["Text"] =  com_text_values
fetched_data

,Date,Text
0,4月8日 10:04,予定より早く利食いました。皆さん、お世話になりました。ありがとうございました。利食いを早めた...
1,4月8日 09:39,おやっ、今日は何かいいことあったん？
2,4月8日 01:08,だから俺が言ったじゃん。\nオービックは割高だって。案の定、俺が書き込みした後１０％近く下が...
3,4月8日 00:22,ここは株価が先取りしすぎて、どんなに業績がよくても、なかなか上がりませんね。含み損がいよいよ...
4,4月7日 17:52,20日線の、18000まで、\n下がる可能性が、\n少しありそう🤏
...,...,...
208,2月4日 12:45,当社はいい材料がないのよ\n低迷続くよ
209,2月4日 11:44,うわ〜、ちょっと目を離したらもう一つ刺さってた\n今日はここまでにして\n出かけてくる
210,2月4日 10:02,後2000は下がる\n株価が高すぎる
211,2月4日 09:49,わぅ、約定した！\n喜ぶべきか、悲しむべきか


試しに保存もしてみる(**2.6 取得したメッセージデータをpandasDataFrameに格納し、csv形式で保存する**)

In [26]:
import os
#Sampleフォルダの作成
if not os.path.exists(os.getcwd()+  "/Sample") :
    os.mkdir(os.getcwd()+  "/Sample")

import pandas as pd
fetched_data.to_csv('/content/Sample/fetched_data.csv')
df = pd.read_csv('/content/Sample/fetched_data.csv', index_col=0)
df

,Date,Text
0,4月8日 10:04,予定より早く利食いました。皆さん、お世話になりました。ありがとうございました。利食いを早めた...
1,4月8日 09:39,おやっ、今日は何かいいことあったん？
2,4月8日 01:08,だから俺が言ったじゃん。\nオービックは割高だって。案の定、俺が書き込みした後１０％近く下が...
3,4月8日 00:22,ここは株価が先取りしすぎて、どんなに業績がよくても、なかなか上がりませんね。含み損がいよいよ...
4,4月7日 17:52,20日線の、18000まで、\n下がる可能性が、\n少しありそう🤏
...,...,...
208,2月4日 12:45,当社はいい材料がないのよ\n低迷続くよ
209,2月4日 11:44,うわ〜、ちょっと目を離したらもう一つ刺さってた\n今日はここまでにして\n出かけてくる
210,2月4日 10:02,後2000は下がる\n株価が高すぎる
211,2月4日 09:49,わぅ、約定した！\n喜ぶべきか、悲しむべきか


ここまでで、

~~1. 取得したい企業のスレッドにアクセスする~~

~~2. 最下部までメッセージをスクロールする~~

~~3. クローリングでメッセージデータを取得する~~

4. 全てのメッセージを取得したら次のスレッドに行く

5. 1~4を繰り返し、取得したい最初の日付から最後に日付までのデータを取得できたらプログラムを終了する

~~6. 取得したメッセージデータをpandasDataFrameに格納し、csv形式で保存する~~

のそれぞれの機能ができました。ここから4, 5を作成し、1〜6を組み合わせます！


**2.4全てのメッセージを取得したら次のスレッドに行く**

In [27]:
import time

from selenium import webdriver



# Google Drive起動
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

# yahoo表示
driver.get('https://finance.yahoo.co.jp/cm/message/1004684/a5aaa1bca5sa5ca5af')

url = driver.find_element_by_class_name("threadBefore").find_element_by_tag_name("a").get_attribute("href")
time.sleep(3)
driver.get(url)
# WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))

# yahoo表示
# https://a-zumi.net/python-selenium-find-element-by-link-text/


#操作メソッド　
# https://qiita.com/memakura/items/20a02161fa7e18d8a693#%E3%82%AF%E3%83%AA%E3%83%83%E3%82%AF%E3%81%99%E3%82%8B

for txt in driver.find_elements_by_class_name("comText") :
  print(txt.text)

同じCMをいつまでも流す嫌味な会社
ありがとう、確かに空売りしてる人少ないね
信用の貸借倍率だよ
これって何の倍率なん？
倍率をみれば　買うのは　とにかく　今じゃない
61倍
ここは、毎日株価をチェックするような銘柄じゃないと思う。数ヶ月とか数年単価で大きくなっていくイメージ。

短期での値幅求めるなら、さっと他に切りかえた方がストレスフリーかもしれない。

とりあえず、半年後位目処に、新高値更新を。
動きません…泣
少し、上向きに動くのか？
今日こそ動いてほしい。
オービックリ…笑笑
そうなれば嬉しい♡
今日はオービックリの2万円台復帰でお願いします
オービックって無借金経営だろ
株価　株主対策要らねえわ
上昇トレンドに転換するまで一か月かかる
二度と投資しない銘柄
目標株価　　17650円
2000円の割高
終わってる　魅力ない企業
割高やね（笑）俺の邪道指数では15,000円です?????沖縄もクラスターでたーいへんです❢
寿命が短い…思い当たります。笑
キリンさん、面白いですね。
プラテンしたよ！
あと１０００円位落ちないと

本格的に反転はできないんじゃないの？
悲観的だと寿命が短いらしいよ
コップに半分しか水がない　×
コップに半分も水が残ってる　◯


**2.5 1~4を繰り返し、取得したい最初の日付から最後に日付までのデータを取得できたらプログラムを終了する**


<<やる流れのメモ>>

1. 取得したい日付を指定 最初と最後を指定
 -   datetime型に -> 変換

2. スレッドの最初を driver.find　で取得
 - datetime型に -> 変換
 
3. 指定した日付とfindした日付を比較
  - end_dateを比較: 取得したい終わり日より現在参照しているスレッドの最初の日の大きい場合に、データの取得を開始する
  - start_dateを比較: 指定した最初の取得日が現在見ているスレッドの最大より大きい場合に、データの取得を終了する


In [2]:
import datetime
# 1. 日付を指定 最初と最後を指定
#  -   datetime型に -> 変換
end_date = datetime.datetime(year=2021, month=3, day=5)
start_date = datetime.datetime(year=2021, month=1, day=5)

if start_date > end_date :
  print("dateに問題があります")

else :
  print("正常にdateを設定しました")

正常にdateを設定しました


In [9]:
current_thread_first = datetime.datetime(year=2021, month=2, day=5)

#現在参照しているスレッドの最初の日よりend_dateが大きい場合: trueの場合は欲しいデータがスレッド内にある
current_thread_first < end_date

#スレッドの最初⇩
#   　　　　　　　　　    |----|
#      |----------|
#     enddate↑取得したいデータの日付の最後の日

True

In [10]:
current_thread_first = datetime.datetime(year=2020, month=11, day=5)

#指定した最初の取得日が現在見ているスレッドの最大より大きい場合: trueの場合は欲しいデータを取り終えるため、データの取得を終了する
current_thread_first < start_date

# スレッドの最後⇩
#   　　　      　 |---|
#             |----------|
#  start_date↑取得したいデータの日付の最初の日

True

# 実際に組み合わせてみる

In [28]:
#seleniumのインストール
# reference https://enjoy-a-lot.com/google-colaboratory-selenium/#toc1
!pip install selenium

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [106]:
from selenium import webdriver

# Google Driver起動
def generate_dirver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=options)
  return driver

#--------------
def fetch_inc_name(driver) -> str:
  date_thread = driver.find_element_by_xpath('//a[contains(@href,"%s")]' % date_url).text
  inc_name = date_thread.split()[2]
  return inc_name

#--------------
def scroll_webpage(driver) -> None:
  # javascriptを実行してページの最下部へ移動
  # https://teratail.com/questions/122242
  html01=driver.page_source
  count = 0
  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(3)
      count += 1
      print(count)
      html02=driver.page_source
      if html01!=html02:
          html01=html02
      else:
          break
#--------------
def move_before_thread_url(driver) -> None:
  url = driver.find_element_by_class_name("threadBefore").find_element_by_tag_name("a").get_attribute("href")
  time.sleep(3)
  driver.get(url)
  

##---------------
import re
import pandas as pd

def fetch_thread_com_values(driver) -> pd.DataFrame: 
  com_text_values = []
  com_date_values = []

  #コメントのテキストを取得
  com_text = driver.find_elements_by_class_name("comText")
  #コメントと日付を取得
  for idx in range(len(com_text)):
    #Date
    tmp = driver.find_elements_by_class_name("comment")[idx].find_elements_by_tag_name("a")
    count = 0
    for val in tmp :
      count += 1
      if len(re.findall('.*月.*日*', val.text)) : 
        #Dateのvalをappend
        com_date_values.append(val.text)
        #Textのvalをappend
        com_text_values.append(com_text[idx].text)
        break
    if count == len(tmp):
      # "コメントは非表示対象"というものがあった場合
      print(list(map(lambda x : x.text , tmp)))
    
  print(f"com_text_values len: {len( com_text_values )}")
  print(f"com_date_values len: {len( com_date_values )}")
  fetched_data = pd.DataFrame()
  fetched_data["Date"] =  com_date_values
  fetched_data["Text"] =  com_text_values
  return fetched_data
  # return com_text_values

#--------------

def extract_data_by_date(fetched_data: pd.DataFrame, start_date: datetime, end_date: datetime) -> pd.DataFrame: 

  #　微調整する
  idx = 0
  for idx in range(len(fetched_data["Date"].values)):
    if "年" not in fetched_data["Date"].values[idx] :
      fetched_data["Date"].values[idx] = "2021年" + fetched_data["Date"].values[idx]
 
  func = lambda x: datetime.datetime.strptime(x, '%Y年%m月%d日 %H:%M%S')
  fetched_data["Date"] = fetched_data["Date"].map(func)
  
  #必要な日付だけを抽出して保存する
  #https://qiita.com/mSpring/items/6ec1ab28dcb261db2c73
  extracted_data = fetched_data[(fetched_data["Date"] >= start_date) & (fetched_data["Date"] <= end_date) ]
  return extracted_data

import os
def save_extracted_data_to_csv(extracted_data: pd.DataFrame, inc_name: str) -> None :
  #Sampleフォルダの作成
  if not os.path.exists(os.getcwd()+  "/Sample") :
    os.mkdir(os.getcwd()+  "/Sample")
  extracted_data.to_csv(f'/content/Sample/{inc_name}.csv')

def read_csv_data(inc_name):
  return pd.read_csv(f'/content/Sample/{inc_name}.csv', index_col=0)

In [77]:
import datetime
# 1. 日付を指定 最初と最後を指定
#  -   datetime型に -> 変換
end_date = datetime.datetime(year=2021, month=3, day=5)
start_date = datetime.datetime(year=2021, month=1, day=5)

if start_date > end_date :
  print("dateに問題があります")

else :
  print("正常にdateを設定しました")

#ある企業の最新のスレッド
start_url = "https://finance.yahoo.co.jp/cm/message/1835700/469ffa7e39f26681161f1f90b137072c/2"

正常にdateを設定しました


テストデータ

- オービック
 - https://finance.yahoo.co.jp/cm/message/1004684/a5aaa1bca5sa5ca5af 
 読むにはここをクリックしてください
- SANEI	
 - https://finance.yahoo.co.jp/cm/message/1835669/8338eaef61ed9597989a053bce4a1891
- ファンペップ	
 - https://finance.yahoo.co.jp/cm/message/1835646/51c5d7fbb91b959380e70d6d9fa22cb7
- 東和ハイシステム	
  - https://finance.yahoo.co.jp/cm/message/1835700/469ffa7e39f26681161f1f90b137072c/2
- クリングルファーマ
 - https://finance.yahoo.co.jp/cm/message/1835646/c64343539df291f82976a870a3ac7d74/2

In [79]:
#自動操作をできるようにする
driver = generate_dirver()

# 注意　

このプログラムは入れたURLが最新のスレッドであることを前提に組まれています

In [80]:
import time
import re

#yahoofinanceにアクセス 企業ごとに設定
driver.get(start_url)

#aタグで見つけるためのモノ
date_url = "https://finance.yahoo.co.jp/cm/message"

#csvデータを取得
inc_name = fetch_inc_name(driver)

#最初に取得したものが最新であるという前提
fetched_time = 0 

#endとstartが含まれているデータを取得済みか判断するフラグ
is_end_data_fetched = False
is_start_data_fetched = False

# 取得したDataFrame型の値を入れるモノ
fetched_data_values = pd.DataFrame()
fetched_data_values["Date"] =[]
fetched_data_values["Text"] = []


#---------------
#ここで見るべきは、スレッドの日付よりend_dateが大きいかを見る
#->前提として途中はなく常に最新のスレッドを起点にやる
while not is_end_data_fetched :
  fetched_time += 1
  #スレッドのタイトルの日付を取得 (例: 4684 - (株)オービック 2021/02/04〜)
  date_thread = driver.find_element_by_xpath('//a[contains(@href,"%s")]' % date_url).text
  print(date_thread)
  #取得したtheadから日付データを示すものに限定させる。  返り値例-> ex1) ['2021/02/04']   ex2) ['2020/01/23', '2021/02/03']
  tmp = list(filter(lambda x : 1 == len(re.findall('.*/.*/*', x)), list(filter(lambda x : "〜" in x ,  date_thread.split()))[0].split("〜")))
  #tmpの0番目のデータを取得
  date_before_tmp = datetime.datetime.strptime(tmp[0], '%Y/%m/%d')
  #　取得したい終わり日より現在参照しているスレッドの最初の日の大きい場合
  if end_date <= date_before_tmp :
    #データの取得をしない
    #最古の場合
    if fetched_time != 1 and len(tmp) == 1:
      print(f"指定した企業：　{inc_name} はあなたが指定した日付のコメントデータを保持していませんでした。　end_date: {end_date}")
      break
   
  #取得できる場合(#　取得したい終わり日より現在参照しているスレッドの最初の日が小さい場合)
  else:
    #df = 取得する関数を呼び出す。
    scroll_webpage(driver)
    # test = fetch_thread_com_values(driver) 
    fetched_data = fetch_thread_com_values(driver) 
    fetched_data_values = pd.concat([fetched_data_values, fetched_data])
    is_end_data_fetched = True
    break
  
  #スレッドの移動
  url = driver.find_element_by_class_name("threadBefore").find_element_by_tag_name("a").get_attribute("href")
  time.sleep(3)
  driver.get(url)


#---------------
if not is_end_data_fetched :
  # end_dataを取得できなかった場合(スレッドに欲しいdataがなかった場合)
  pass

#TODO 21;00 
else:
  while not is_start_data_fetched:

    #スレッドの移動
    url = driver.find_element_by_class_name("threadBefore").find_element_by_tag_name("a").get_attribute("href")
    time.sleep(3)
    driver.get(url)

    #スレッドのタイトルの日付を取得 (例: 4684 - (株)オービック 2021/02/04〜)
    date_thread = driver.find_element_by_xpath('//a[contains(@href,"%s")]' % date_url).text
    print(date_thread)
    #取得したtheadから日付データを示すものに限定させる。  返り値例-> ex1) ['2021/02/04']   ex2) ['2020/01/23', '2021/02/03']
    tmp = list(filter(lambda x : 1 == len(re.findall('.*/.*/*', x)), list(filter(lambda x : "〜" in x ,  date_thread.split()))[0].split("〜")))
    
    #最古のスレッドに到達したかどうか
    if len(tmp) == 1:
      date_after_tmp = datetime.datetime.strptime(tmp[0], '%Y/%m/%d')
    else:
      date_after_tmp = datetime.datetime.strptime(tmp[1], '%Y/%m/%d')

    #指定した最初の取得日が現在見ているスレッドの最大より大きいか
    if start_date < date_after_tmp:
      scroll_webpage(driver)
      fetched_data = fetch_thread_com_values(driver) 
      fetched_data_values = pd.concat([fetched_data_values, fetched_data])
      # test = fetch_thread_com_values(driver) 

    #最初の取得日を含むデータを既に取得している場合
    else :
      print(f"指定した企業：　{inc_name} の指定した日付のコメントデータを取得しました。　start_date: {start_date}")
      is_start_data_fetched = True

driver.close()
print("-"*100)
fetched_data_values

4172 - 東和ハイシステム(株) 2020/12/29〜
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
com_text_values len: 433
com_date_values len: 433
4172 - 東和ハイシステム(株) 〜2020/12/28
指定した企業：　東和ハイシステム(株) の指定した日付のコメントデータを取得しました。　start_date: 2021-01-05 00:00:00
----------------------------------------------------------------------------------------------------


,Date,Text
0,4月8日 17:59,いつになったら上がるのかねー\nてか何でも良いから出来高欲しいわ。
1,4月8日 08:04,コロナ追い風
2,4月7日 11:39,月曜日の上げ分全て無駄に😞\n積極的な情報発信で知名度を上げないと厳しいかな。
3,4月7日 11:03,毎日よー降ってくるのぉ～
4,4月6日 09:25,底堅い動きで監視。前回前々回より支持線弱し。次に期待か
...,...,...
428,2020年12月29日 08:09,しゃああああああああああ\n\n控えめに言って割高で買いが入りません\n\n大げさに言ったら...
429,2020年12月29日 07:49,なんか取り敢えずIPO出たら全部の名前言って、外れた奴は無かったことにして一切投稿せず、当た...
430,2020年12月29日 07:23,ウケたー\n>kkさんは明日GDからの3700円くらいで、しゃあああああザラ場高値ブレイク！...
431,2020年12月29日 06:59,ありそうで草\n3700になっても誰も救われないが彼はバーチャだから気づけないだろう


In [108]:
extracted_data = extract_data_by_date(fetched_data, start_date, end_date) 
save_extracted_data_to_csv(extracted_data, inc_name) 
data = read_csv_data(inc_name)
data

,Date,Text
37,2021-03-04 12:04:06,再ＩＮ(^ω^)\nちょっと売られ過ぎ。
38,2021-03-04 12:03:00,春だな(´・ω・`)\n株価も下がってきたね。\n\nさて、何度も何度も春だと書いてきたけれ...
39,2021-03-03 11:05:09,日経cnbc！
40,2021-03-03 11:00:01,春の珍事も終わりか(´・ω・`)\nま、春だからねぇ…\n春かぁ…
41,2021-03-03 10:01:07,地味だけど悪くないwww\n中長期で持てばいいなwww
...,...,...
279,2021-01-05 11:00:04,ここもちょいちょい見とくかな(｡･ω･?)
280,2021-01-05 10:00:08,買い板3510―5300株 2件の5000株が消えた、見せ板か。
281,2021-01-05 09:03:07,もう少し増したいがなかなか落ちてこないべ、この辺が底でねーの？\n抜けた歯が埋まったか（笑）
282,2021-01-05 09:00:09,昨日の安値近辺でイン予定
